In [0]:
#!pip install telebot
#!pip3 uninstall telebot
!pip3 install pytelegrambotapi --upgrade > /dev/null 
!pip install pymorphy2 > /dev/null

In [117]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

import telebot
import random
import time

from sklearn.metrics.pairwise import linear_kernel

import string
nltk.download('punkt')
nltk.download('stopwords')
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
tfidf = TfidfVectorizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
stickers_ids = ['CAACAgIAAxkBAAN2XohnGn0E2PpnDa8sLRjIfoXJkMgAAucCAAJjK-IJgZ3pFN1v78MYBA',
'CAACAgIAAxkBAAN6XohnHcqqlO0fWzeP5hyVEW-Q0tUAAswCAAJjK-IJzL8QiJ5hkasYBA',
'CAACAgIAAxkBAAODXohnJLq7B2-qL_YDJwpvUNp60EAAAqgCAAJjK-IJ0Knq1taB2AEYBA',
'CAACAgIAAxkBAAOHXohnJjVasUvm0at_qVDuO7AN_s8AAqICAAJjK-IJDEdBkfGfrdYYBA',
'CAACAgIAAxkBAAOJXohnJ9H6Ra8cxM5szXoW1j0AAa5_AAKcAgACYyviCVvc75FsiA97GAQ',
'CAACAgIAAxkBAAN_XohnIa7dO6ISHOtT8iIWr69aZJ8AAroCAAJjK-IJbneXMn3Iye8YBA', 
'CAACAgIAAxkBAAN-XohnIN7xjPDIDGCH5LWx4LkiGdUAArwCAAJjK-IJ-4zaoKwb82cYBA', 
'CAACAgIAAxkBAAN0XohnGEd0ZCsu3v7ZVow44enNBUAAAtoCAAJjK-IJauzw6tbgTlgYBA', 
'CAACAgIAAxkBAAN1XohnGfEueVTYaiDyaJXZzknDzyYAAtgCAAJjK-IJ0c93YMe3wtsYBA', 
'CAACAgIAAxkBAAN3XohnGsaRKK2tQHUikHhW3MVNxKsAAtYCAAJjK-IJscEHEE2deiEYBA', 
'CAACAgIAAxkBAAN4XohnG3HoKJhy_rjeND70rx9-29wAAsYCAAJjK-IJzec08mzFJQ4YBA', 
'CAACAgIAAxkBAAN5XohnHfV86f6rIs8PFAOuTzu4hk4AAsoCAAJjK-IJQ-opywa_2tMYBA', 
'CAACAgIAAxkBAAN7XohnHphjjO53yx1rXeW7dfX0BQ0AAtICAAJjK-IJUJFHXqol5P0YBA', 
'CAACAgIAAxkBAAN9XohnIOVsBaftETwrCTzgMY3P-uQAAsQCAAJjK-IJfFM_LRL30PMYBA', 
'CAACAgIAAxkBAANoXohnEGpDZr7m28TSltOJfdliPssAAhQDAAJjK-IJaxaD98tkXc8YBA', 
'CAACAgIAAxkBAANpXohnEkM5ZS-eeuVIjVpEs9SFm-kAAhIDAAJjK-IJkw6POAaUJz0YBA', 
'CAACAgIAAxkBAANqXohnE--5DGSMCm1Xvqqfmi4VjysAAg4DAAJjK-IJF3ZWXqnR2U4YBA', 
'CAACAgIAAxkBAANrXohnE1BGcHiL9ChQ9IhwOPi0zbYAAgwDAAJjK-IJVBdFLu9Z334YBA', 
'CAACAgIAAxkBAANsXohnFHhivGP6HdfRQ8KbROuWZmEAAggDAAJjK-IJ3pWODHATQt0YBA', 
'CAACAgIAAxkBAANuXohnFYx7Wu3QYUM1kNVfUrXQW3IAAt4CAAJjK-IJkEJR_BL_n5MYBA', 
'CAACAgIAAxkBAANvXohnFpGeKuTvxJfV8l3oOVlPvO0AAuMCAAJjK-IJxoTo3R-Dn4kYBA', 
'CAACAgIAAxkBAANwXohnFqrdM5JEVWsBkXEofbeQ-jEAAuwCAAJjK-IJ1ybfVvDJmr8YBA', 
'CAACAgIAAxkBAANyXohnFxrgfKFYdPhFBxqzzwO2bjYAAgIDAAJjK-IJM1tbM7mpBCsYBA', 
'CAACAgIAAxkBAANzXohnGJJaNnPDbxvZh56kQX8fr9IAAtwCAAJjK-IJS8TCPIGDi8kYBA', 
'CAACAgIAAxkBAAOAXohnInsqNljONwUy9Wf2IhngslAAArgCAAJjK-IJO9-36Nk_9JsYBA', 
'CAACAgIAAxkBAAOBXohnIlD94WdP7bz2sZ1FNQcOgloAArICAAJjK-IJXtqn3ejnpP0YBA', 
'CAACAgIAAxkBAAOCXohnI8cuZa9UhsOJjL770GdURYgAAqQCAAJjK-IJekxACxNzwcQYBA', 
'CAACAgIAAxkBAAOEXohnJPK1QPhDT1zTW_IV1LN-mrgAAqoCAAJjK-IJdjaSdZSUJbYYBA', 
'CAACAgIAAxkBAAOFXohnJVj2igpN_C_wLS8r5pRYUPoAAqwCAAJjK-IJeBtXRF_LpwsYBA', 
'CAACAgIAAxkBAAOGXohnJczgrPUt_QFTDBqym7zHRbsAAq4CAAJjK-IJJUGyNW_KPTEYBA', 
'CAACAgIAAxkBAAOIXohnJ8Wr-DyuC6tvStjtZJ2WYRkAAqACAAJjK-IJ5eT-NU0XyOAYBA', 
'CAACAgIAAxkBAAOKXohnKM0206ZKxdgmOb8KaAbZ4QsAApoCAAJjK-IJI1GLrSIwyv0YBA', 
'CAACAgIAAxkBAAOLXohnKNeIj-VbxTLW8g8zWBY3Hv4AApYCAAJjK-IJTfSNA206rU4YBA', 
'CAACAgIAAxkBAAOMXohnKmfPVWS3KGEVU_3H59Y959oAApgCAAJjK-IJZadb5UJwT84YBA', 
'CAACAgIAAxkBAAOOXohnK6HTXiY2gzOsvRBIjYKKJVoAAhYDAAJjK-IJ_jzqyKdDpjsYBA', 
'CAACAgIAAxkBAAOPXohnLMMSLd0plo22Id73oxkdT5AAAp4CAAJjK-IJxptc4DRhmEQYBA', 
'CAACAgIAAxkBAAOQXohnLa5XUAsjiPePFeRC8oeAqDEAAh0GAAJjK-IJOM2IMvQWyocYBA', 
'CAACAgIAAxkBAAN8XohnHnnwHrkKy9KMMnfDf_vyQqMAAtQCAAJjK-IJEaAVfvqwVXoYBA', 
'CAACAgIAAxkBAANtXohnFaYStFiZQOlvY_p_vap0WkMAAgQDAAJjK-IJ5Bhot_PLURcYBA', 
'CAACAgIAAxkBAAONXohnKgbBchQUV-4XGEwus7cL1-sAAuECAAJjK-IJ-1wpHc7C8HgYBA', 
'CAACAgIAAxkBAANxXohnF1wbHy_l4IlMWx4qGThEkYcAAwMAAmMr4gk_-9kTKMo6YBgE']

In [0]:
questions = ['Какой фильм тебе нравится?' , 'Дай мне название какого-нибудь фильма.']
answers = ['Ты продолжай, когда я тебя пойму, я тебе отвечу...' , 'Давай, накидывай, я разберусь...' , 'Тебя родные хоть понимают?']

In [0]:
my_stop_words = stopwords.words('russian')
my_stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

In [0]:
def tokenize_ru(file_text):
# firstly let's apply nltk tokenization
  tokens = word_tokenize(file_text)

# let's delete punctuation symbols
  tokens = [i for i in tokens if (i not in string.punctuation)]

# deleting stop_words
  # stop_words = stopwords.words('russian')
  # stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])
  tokens = [i for i in tokens if (i not in my_stop_words)]

# cleaning words
  tokens = [i.replace("«", "").replace("»", "") for i in tokens]
  tokens = [ morph.parse(i)[0].normal_form for i in tokens ]
  return str(tokens).strip('[]').replace("'","").replace(",","")

In [0]:
# df = pd.read_csv('movie.csv')

# tfidf = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf.fit_transform(df['common'])

# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# indices = pd.Series(df.index, index=df['original_title']).drop_duplicates()

In [0]:
df_top250 = pd.read_csv('top250.csv', sep='#' , names=['original_title','year', 'country' , 'score' , 'overview','director','screenplay' , 'actors' , 'url'])
df_top250['overview_tokenize'] = df_top250['overview'].map(lambda x: tokenize_ru(str(x)))
df_top250['original_title_lower'] = df_top250['original_title'].map(lambda x: x.lower().strip())
df_top250['creators'] = df[df.columns[5:8]].apply(
    lambda x: ','.join( x.dropna().astype(str)),
    axis=1).apply(lambda x: str(x).replace(" ","").replace(","," "))

In [118]:
print(df_top250['creators'][13])

КвентинТарантино КвентинТарантино РоджерЭвери ДжонТраволта СэмюэлЛ.Джексон БрюсУиллис УмаТурман ВингРеймз


In [0]:
# for test
tfidf = TfidfVectorizer(stop_words=my_stop_words)
tfidf_matrix = tfidf.fit_transform(df_top250['overview_tokenize'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df_top250.index, index=df_top250['original_title_lower']).drop_duplicates()

In [108]:
"""
tfidf_matrix_overview = tfidf.fit_transform(df_top250['overview_tokenize'])
cosine_sim = linear_kernel(tfidf_matrix_people, tfidf_matrix_people)
indices = pd.Series(df.index, index=df_top250['original_title_lower']).drop_duplicates()
"""

"\ntfidf_matrix_overview = tfidf.fit_transform(df_top250['overview_tokenize'])\ncosine_sim = linear_kernel(tfidf_matrix_people, tfidf_matrix_people)\nindices = pd.Series(df.index, index=df_top250['original_title_lower']).drop_duplicates()\n"

In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    a = df_top250['original_title_lower'].iloc[movie_indices]
    #a = str(a.values).strip('[]').replace("' '" , '\n').replace("'" , "").strip()
    #print(sim_scores)
    return a.values[random.randint(0,9)].capitalize()

In [110]:
get_recommendations('криминальное чтиво')

'Легенда о пианисте'

In [0]:
df_dialogs = pd.read_excel('dialog_talk_agent.en.ru.xlsx', names=['Context','Text Response'])
df_dialogs.ffill(axis = 0,inplace=True)
df_dialogs = df_dialogs.append({'Context' : 'Ты повторяешься' , 'Text Response' : 'Да и ты тоже'} , ignore_index=True)
df_dialogs = df_dialogs.append({'Context' : 'Пойдем пить пиво' , 'Text Response' : 'Да возьми мне пару баночек'} , ignore_index=True)
df_dialogs['lemmatized_text'] = df_dialogs['Context'].map(lambda x: tokenize_ru(str(x)))

In [0]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# stop = stopwords.words('english')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')


x_tfidf=tfidf.fit_transform((df_dialogs['lemmatized_text'])).toarray() 
df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names())

In [0]:
def chat_tfidf(text):
    lemma=tokenize_ru(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df_dialogs['Text Response'].loc[index_value]

In [122]:
chat_tfidf('Давай выпьем по пиву?')

'Да возьми мне пару баночек'

In [0]:
bot = telebot.TeleBot('1171053774:AAEKARUcqloBdKpZyT9_g4jnIKDcOUJYOKU')

def random_movie():
  return df_top250['original_title'][random.randint(0,len(df_top250['original_title'])-1)]
  
def get_button():
  keyboard1 = telebot.types.ReplyKeyboardMarkup(True, True)
  return keyboard1.row(random_movie().capitalize(),random_movie().capitalize(),random_movie().capitalize())
  del keyboard1

def get_elem(elem):
  return elem[random.randint(0, len(elem)-1)]

@bot.message_handler(commands=['start'])
def start_message(message):
  bot.send_sticker(message.chat.id, get_elem(stickers_ids) )
  bot.send_message(message.chat.id, get_elem(questions) , parse_mode="markdown" , reply_markup=get_button() ) 

@bot.message_handler(commands=['film'])
def start_message(message):
  bot.send_sticker(message.chat.id, get_elem(stickers_ids) )
  bot.send_message(message.chat.id, get_elem(questions) , parse_mode="markdown" )

@bot.message_handler(content_types=['text'])
def send_text(message):
    #if message.text.lower().split(' ')[0] == 'rec':
    if message.text.lower() in list(df_top250['original_title_lower']): # if message.text.lower() in list(df['original_title']):
        #telebot.
        #user = update.message.from_user
        bot.send_message(message.chat.id,  str(get_recommendations(str(message.text.lower()  )))) #str(get_recommendations('The Hateful Eight')     )) #    str(get_recommendations('The Hateful Eight')) )
        #bot.send_photo(message.chat.id,  'https://st.kp.yandex.net/images/film_iphone/iphone360_326.jpg' , caption = "Карты"  )
        #print('You talk with user {} and his user ID: {} '.format(user['username'], user['id']))
    elif message.text.lower().split(' ')[0] == 'r':
        bot.send_photo(message.chat.id,  'https://st.kp.yandex.net/images/film_iphone/iphone360_326.jpg' , caption = "Карты"  )
    elif message.text.lower().split(' ')[0] == 'w':
        try:
          content = str(wikipedia.summary(message.text.lower().split(' ')[1:]))
          summary = summarize(content, split=True, word_count=50)
        except BaseException as e:
          summary = "Попробуй еще"
        bot.send_message(message.chat.id, str(summary) )
    elif message.text.lower().split(' ')[0] == 'wiki':
        #content = str()
        #summary = summarize(content, split=True, word_count=100)
        try:
          content = wikipedia.summary(message.text.split(' ')[1:])
        except BaseException as e:
          content = "Попробуй еще"        
        bot.send_message(message.chat.id, content )
    elif message.text.lower() == 'я тебя люблю':
        bot.send_sticker(message.chat.id, 'AAMCAgADGQEAAyleiGTjHWbrAsBTwpvnfoF99txI1QACEgMAAmMr4gmTDo84BpQnPTDycA0ABAEAB20AA8ktAAIYBA')
    else : 
        #bot.send_sticker(message.chat.id, 'CAADAgADZgkAAnlc4gmfCor5YbYYRAI')
        time.sleep(1)
        bot.send_message(message.chat.id, chat_tfidf(message.text.lower()) , reply_markup=get_button())
        if bool(random.choices([True, False], weights=[40, 60])[0]):
          bot.send_sticker(message.chat.id, get_elem(stickers_ids) )
# get stiker id
@bot.message_handler(content_types=['sticker'])
def sticker_id(message):
    print(message.sticker.file_id)


bot.polling()